# Sem Pormpt

In [12]:
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import SystemMessage
import numpy as np
from IPython.display import display, Markdown

In [13]:
client = OpenAI()

In [14]:
#Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
#Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)


#Verificando se foi extraido corretamente
print(split_texts[0])

#Embbeding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

#Retriver
retriever = vector_store.as_retriever()
# Preparar o contexto
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)
#Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
        }
    ]
)


#Resposta
print(completion.choices[0].message.content)


page_content='Adicionar verificação de configuração dos arquivos .xml 
 
PROCEDIMENTO (XXXX) CENTRAL 2.0 
iles de hardware e software da Central 2.0 verificando o perfeito funcionamento da mesma. Na 
sequência, o cadastramento da central no DEVICE MANAGER para controle de estoque.  
A configuração com a aplicação do cliente a ser instalada, junto com informações de rede 
fecham o procedimento (XXXX). 
 
Para informações sobre imagens explodidas da central 2.0 prosseguir ao procedimento forçar 
(XXXX). 
Para informações sobre a montagem da central 2.0 prosseguir ao procedimento (XXXX). 
. 
1.​Pré-teste Versão de Hardware 2.0 
 
Verificar se o GATEWAY está conectado a Raspberry.' metadata={'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'files/Configuração e teste - Central SensorGW.pdf', 'file_path': 'files/Configuração e teste - Central SensorGW.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': 'Configuração e teste - Central SensorGW

# Com Prompt

In [16]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.create_documents(texts)

# Embedding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

# Retriever
retriever = vector_store.as_retriever()

# Pegar os documentos relevantes baseado na pergunta
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)

# Preparar o contexto
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)

# Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", 
         "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}"
         },
        {"role": "user", 
        "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
         } 
    ]
)

# Resposta
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos abaixo:

1. **Acesso ao Putty**:
   - Abra o Putty no seu computador.
   - Conecte-se à central usando o terminal.

2. **Configuração da Central**:
   - Digite o seguinte comando para iniciar a configuração da central para o envio de dados ao portal do cliente:
     ```
     sensorgw appConfig
     ```
   - Quando perguntar "Default server configuration? (y or n)", responda "y" e pressione Enter.
   - Preencha as informações solicitadas:
     - Client WebApp: "TESTE"
     - Client Login: (Digite o login do cliente)
     - Client Password: (Digite a senha do cliente)
     - CheckConnection: (Preencha conforme necessário)
     - CheckModem: (Preencha conforme necessário)
     - Carrier: "operadora" (Selecione a operadora)
   - Após inserir as informações, pressione Enter.
   - Execute os comandos abaixo para finalizar a configuração:
     ```
     sync
     sensorgw stop
     ```

3. **Configuração do Hamachi**:
   - Continuando no Putty, insira os comandos a seguir:
     - Para parar o serviço do Hamachi:
       ```
       /etc/init.d/logmein-hamachi stop
       ```
     - Para remover as configurações prévias do Hamachi:
       ```
       rm -Rf /var/lib/logmein-hamachi/
       ```
     - Para iniciar o serviço do Hamachi:
       ```
       /etc/init.d/logmein-hamachi start
       ```
     - Faça login no Hamachi:
       ```
       hamachi login
       ```
     - Defina o nome da central no Hamachi (substitua "CLIENTE" com o nome do cliente):
       ```
       hamachi set-nick "CLIENTE"
       ```
     - Vincule a conta do Hamachi (substitua "eduardosleal@mcasistemas.com.br" com o e-mail do cliente):
       ```
       hamachi attach eduardosleal@mcasistemas.com.br
       ```

4. **Finalização**:
   - Após configurar o Hamachi, siga as instruções para fechar o gabinete da central, limpar a superfície, inserir os adesivos e finalizar o processo de configuração.

Certifique-se de seguir todas as etapas com atenção para garantir que a central esteja configurada corretamente para o cliente.

# Com Hyde

In [17]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Dividir o texto
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.split_documents(docs)

# Criar embeddings e armazenar em um vetor
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents=split_texts, embedding=embedding_model)
retriever = vector_store.as_retriever()

# Definir a pergunta
pergunta = "Como é faço para configurar a central para o cliente?"
hyde_model = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125", 
    temperature=0
)
hyde_prompt = f"""
Responda à seguinte pergunta de forma detalhada e clara. Seja preciso e forneça uma explicação passo a passo:

Pergunta: {pergunta}

Utilize as informações abaixo para criar uma resposta completa:
"""
resposta_hipotetica = hyde_model.invoke([SystemMessage(content=hyde_prompt)])

# Criar embedding da resposta hipotética para busca mais precisa
resposta_hipotetica_embedding = embedding_model.embed_query(resposta_hipotetica.content)

# Criar embedding da pergunta original para busca mais precisa
pergunta_embedding = embedding_model.embed_query(pergunta)

# Garantir que ambos os embeddings sejam numpy arrays
pergunta_embedding = np.array(pergunta_embedding)
resposta_hipotetica_embedding = np.array(resposta_hipotetica_embedding)

# Combinar os embeddings da pergunta original e da resposta hipotética
combined_embedding = (pergunta_embedding + resposta_hipotetica_embedding) / 2

# Buscar documentos com o embedding combinado
retrieved_docs = vector_store.similarity_search_by_vector(combined_embedding)

# Criar o contexto com os documentos recuperados
context_list = []
for doc in retrieved_docs:
    context_list.append(doc.page_content)

# Juntar os contextos recuperados em uma única string
context = "\n\n".join(context_list)

# Gerar a resposta final com base no contexto recuperado
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado e em formato de passo a passo.\n\n{context}"},
        {"role": "user","content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"}
    ]
)

# Exibir a resposta gerada
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos abaixo:

1. Acesso ao PUTTY:
   - Abra o terminal PUTTY para se conectar à central.

2. Configuração da Central para Envio de Dados ao Portal do Cliente:
   - No terminal PUTTY, digite o seguinte comando e pressione Enter:
     ```
     sensorgw appConfig
     ```
   - Quando a mensagem "Default server configuration?" aparecer, digite 'y' e pressione Enter.
   - Será solicitado que você configure algumas informações, como o nome da aplicação do cliente, login do cliente, senha do cliente, entre outros. Siga as instruções na tela e insira os dados necessários.
   - Após configurar todos os dados, execute os seguintes comandos no terminal PUTTY e pressione Enter após cada um:
     ```
     sync
     sensorgw stop
     ```

3. Configuração do Hamachi:
   - Ainda no terminal PUTTY, execute os seguintes comandos para configurar o Hamachi:
     - Parar o serviço do Hamachi:
       ```
       /etc/init.d/logmein-hamachi stop
       ```
     - Remover configurações prévias do Hamachi:
       ```
       rm -Rf /var/lib/logmein-hamachi/
       ```
     - Iniciar o serviço do Hamachi:
       ```
       /etc/init.d/logmein-hamachi start
       ```
     - Fazer o login no Hamachi:
       ```
       hamachi login
       ```
     - Definir o nome da central para o Hamachi (substitua 'CLIENTE' pelo nome desejado):
       ```
       hamachi set-nick “CLIENTE”
       ```
     - Vincular a conta do Hamachi (substitua 'eduardosleal@mcasistemas.com.br' pelo e-mail apropriado):
       ```
       hamachi attach eduardosleal@mcasistemas.com.br
       ```

4. Configuração das Informações de Rede da Central:
   - Defina as informações de rede da central para IP Dinâmico (DHCP).
   - Acesse o navegador web e digite o IP da central.
   - Faça login com as credenciais de teste (Login: teste, Senha: teste).

5. Testes Finais:
   - Teste se a bateria mantém o hardware em funcionamento sem a fonte, desconectando a fonte da central.
   - Teste o botão ON/OFF da Central, verificando se é possível desligar a central com um clique e forçar o desligamento mantendo o botão pressionado.

Após seguir esses passos, a central estará configurada para o cliente e pronta para operação.